# Qubit-data for Lithium Hydride

In [1]:
import pickle
from pylab import *
import numpy as np
import math as m
from functools import reduce
import matplotlib.pyplot as plt

## Preliminaries

The fermionic Hamiltonian in second quantization (with $M$ orbitals)

$$ H = \sum_{\alpha,\beta=1}^M t_{\alpha\beta}c^\dagger_\alpha c_\beta 
+ \frac{1}{2}\sum_{\alpha\beta=1}^M\sum_{\gamma\delta=1}^M u_{\alpha\beta\gamma\delta}c^\dagger_\alpha c^\dagger_\gamma c_\delta c_\beta$$

is mapped into a qubit Hamiltonian in the form

$$ H = \sum_kc_kP_k $$

where $c_k$ are interaction coefficients and $P_k$ are elements of the $N$-qubit Pauli group $\mathcal{P}_N=\{\mathbb{1},X,Y,Z\}^{\otimes N}$.

The interaction coefficients and the Pauli operators can be found in the `interaction.file` and the `pauli.file` respectively.

Specifics:
- Number of qubits: 4
- Molecule geometry: bond distance
- Fermionic basis: atomic STO-3G
- Qubit mapping: spin-parity

## Exact diagonalization

In [2]:
# Identity 
I = np.asarray([[1.,0.],[0.,1.]])
# Pauli X
X = np.asarray([[0.,1.],[1.,0.]])
# Pauli Y
Y = np.asarray([[0.,-1j],[1j,0.]])
# Pauli Z
Z = np.asarray([[1.,0.],[0.,-1.]])

In [3]:
def operatorfromstring(pauli_string):
    """
    Generate a many-body operator from a list of single-qubit 
    pauli matrices.
    ---------------------------------------------------------
    Input    pauli_string: str , (ex: [X,Z,X,Y,...])
    
    Output   pauli_op: np.array, shape = [2**N,2**N]
    """
    op_list = []
    for k in range(len(pauli_string)):
        if (pauli_string[k] == 'X'):
            op_list.append(X)
        elif (pauli_string[k] == 'Y'):
            op_list.append(Y)
        elif (pauli_string[k] == 'Z'):
            op_list.append(Z)
        else:
            op_list.append(I)

    return reduce(np.kron,op_list)

def hamiltonian(pauli_list,interactions):
    """
    Generate a many-body hamiltonian
    ---------------------------------------------------------
    Input    pauli_list: list of pauli strings
             interactions: list of interaction strengths
    
    Output   hamiltonian: np.array, shape = [2**N,2**N]
    """
    N = len(pauli_list[0])
    hamiltonian = np.zeros((1<<N,1<<N),dtype=complex)
    for i,pauli in enumerate(pauli_list):
        hamiltonian += interactions[i]*operatorfromstring(pauli)
    return hamiltonian

def eigensolve(hamiltonian):
    """
    Compute ground state energy and wavefunction
    """
    (eigenvalues,eigenstates) = np.linalg.eigh(hamiltonian)
    return eigenvalues[0],eigenstates[:,0]

In [4]:
pauli_list   = np.load('paulis.file',allow_pickle=True)
interactions = np.load('interactions.file',allow_pickle = True)
for k in range(10):
    print('Pauli:',pauli_list[k],' | Interaction: %.3E' % interactions[k])
print('...')

Pauli: IIII  | Interaction: -8.022E+00
Pauli: ZIII  | Interaction: -9.276E-02
Pauli: XZII  | Interaction: -2.941E-03
Pauli: XIII  | Interaction: 2.941E-03
Pauli: ZZII  | Interaction: -2.119E-01
Pauli: YYII  | Interaction: 7.428E-04
Pauli: XXII  | Interaction: -7.428E-04
Pauli: ZXII  | Interaction: 1.939E-02
Pauli: IXII  | Interaction: 1.939E-02
Pauli: IZII  | Interaction: 3.653E-01
...


In [5]:
# Build the full Hamiltonian and get the ground state energy / wavefunction
H = hamiltonian(pauli_list,interactions)
E0,psi0 = eigensolve(H)
print('Ground state energy: %.10f\n' % E0 )
print('Ground state wavefunction:\n\n',psi0)

Ground state energy: -8.9073963113

Ground state wavefunction:

 [-4.96394908e-03+0.j  2.49800181e-16+0.j -5.76276175e-02+0.j
  1.11651339e-01+0.j -3.26381702e-02+0.j  5.08356315e-16+0.j
  3.54539651e-02+0.j -5.76276175e-02+0.j  1.38777878e-16+0.j
  1.23592416e-25+0.j  5.01265465e-18+0.j -1.14084922e-17+0.j
 -9.88663405e-01+0.j -2.63778775e-17+0.j -3.26381702e-02+0.j
 -4.96394908e-03+0.j]


 ## Neural-network reconstruction

Quantum state reconstruction of the LiH ground state with a restricted Boltzmann machine, using the NetKet library.

References:
- _"Machine learning quantum states in the NISQ era"_ : review of quantum state reconstruction with restricted Boltzmann machines. https://arxiv.org/abs/1905.04312
- _"Precise measurement of quantum observables with neural-network estimators"_ RBM reconstruction of molecular ground states. https://arxiv.org/abs/1910.07596
- _"NetKet: A Machine Learning Toolkit for Many-Body Quantum Systems"_ Overview of the NetKet software library. https://arxiv.org/abs/1904.00031

In [6]:
from mpi4py import MPI 
import netket as nk

In [7]:
# Unitary matrices for the rotation in the X and Y bases
rotationX = 1./(math.sqrt(2))*np.asarray([[1.,1.],[1.,-1.]])
rotationY= 1./(math.sqrt(2))*np.asarray([[1.,-1j],[1.,1j]])

def LoadData(N,hilbert,path_to_samples, path_to_bases):
    training_samples = []
    training_bases = []

    tsamples = np.loadtxt(path_to_samples)
    assert(N == tsamples.shape[1])
    fin_bases = open(path_to_bases, 'r')
    lines = fin_bases.readlines()

    bases = []

    for b in lines:
        basis = ""
        assert(len(b) == N + 1)
        for j in range(N):
            basis += b[j]
        bases.append(basis)
    index_list = sorted(range(len(bases)), key=lambda k: bases[k])
    bases.sort()

    for i in range(len(tsamples)):
        training_samples.append(tsamples[index_list[i]].tolist())

    rotations = []

    tmp = ''
    b_index = -1
    for b in bases:
        if (b != tmp):
            tmp = b
            localop = nk.operator.LocalOperator(hilbert, 1.0)

            for j in range(N):
                if (tmp[j] == 'X'):
                    localop *= nk.operator.LocalOperator(hilbert, rotationX, [j])
                if (tmp[j] == 'Y'):
                    localop *= nk.operator.LocalOperator(hilbert, rotationY, [j])

            rotations.append(localop)
            b_index += 1
        training_bases.append(b_index)

    return tuple(rotations), np.asarray(training_samples), np.asarray(training_bases)

def OperatorFromString(op_string):                                                
    OpList = []                                                                   
    Sites = []                                                                    
    for k in range(len(op_string)):                                               
        if (op_string[k] == 'X'):                                                 
            OpList.append(X)                                                      
            Sites.append(k)                                                       
        elif (op_string[k] == 'Y'):                                               
            OpList.append(Y)                                                      
            Sites.append(k)                                                       
        elif (op_string[k] == 'Z'):                                               
            OpList.append(Z)                                                      
            Sites.append(k)                                                       
    return Sites,reduce(np.kron,OpList) 

def BuildHamiltonian(N,hilbert,pauli_path,interactions_path):                     
    pauli = np.load(pauli_path,allow_pickle=True)                                 
    interactions = np.load(interactions_path,allow_pickle=True)                   
                                                                                  
    hamiltonian = nk.operator.LocalOperator(hilbert, 0.0)                                                                                 
    for h in range(0,len(pauli)):                                                                                           
        flag = 0                                                                  
        for j in range(N):                                                        
            if (pauli[h][j]!='I'): flag = 1; break                                
        if flag == 0:                                                             
            hamiltonian += nk.operator.LocalOperator(hilbert,np.real(interactions[h]))
        else:                                                                     
            sites,operator = OperatorFromString(pauli[h])                         
            h_term = interactions[h]*operator
            hamiltonian += nk.operator.LocalOperator(hilbert,h_term,sites)        
            
    return hamiltonian 

In [8]:
N = 4          # Number of qubits
ns = 100000    # Number of training samples
lr = 0.01      # Learning rate
M = 10000      # Number of samples for the negative CD phase
bs = 1000      # Batch size
epochs = 500   # Epochs

samples_path = "train_samples.txt"
bases_path = "train_bases.txt"
pauli_path = "paulis.file"
interactions_path = "interactions.file"

In [9]:
# Lattice
graph = nk.graph.Hypercube(length=N, n_dim=1,pbc=False)
# Hilbert space
hilbert = nk.hilbert.Qubit(graph=graph)
# NetKet Hamiltonian
hamiltonian = BuildHamiltonian(N,hilbert,pauli_path,interactions_path)

In [10]:
# Load training data
rotations, tr_samples, tr_bases = LoadData(N,hilbert,samples_path, bases_path)
if (ns > tr_samples.shape[0]):
    "Not enough training samples"
else:
    training_samples = tr_samples[0:ns]
    training_bases   = tr_bases[0:ns]

In [11]:
# RBM wavefunction
#ma = nk.machine.RbmSpinPhase(hilbert=hilbert, alpha=1)  # Real-weights RBM (amplitude + phase)
ma = nk.machine.RbmSpin(hilbert=hilbert, alpha=1)        # Complex-weights RBM, alpha=num_hidden/N
ma.init_random_parameters(seed=1234, sigma=0.01)

In [12]:
# Sampler
sa = nk.sampler.MetropolisLocal(machine=ma)

# Optimizer + Stochastic Reconfiguration
#op = nk.optimizer.Sgd(learning_rate=lr)
op = nk.optimizer.RmsProp(learning_rate=lr,beta=0.9,epscut = 1.0e-6)
sr = nk.optimizer.SR(diag_shift=0.1)

In [13]:
# Quantum state tomography object
qst = nk.Qsr(
    sampler=sa,
    optimizer=op,
    samples=training_samples,
    rotations=rotations,
    bases=training_bases,
    n_samples=M,
    n_samples_data=bs,
    sr=sr,
    #sr=None
)
# Measure the energy while training
qst.add_observable(hamiltonian, "Energy")

In [14]:
# Training loop
for ep in qst.iter(epochs, 10):
    # Get energy measurement
    obs = qst.get_observable_stats()
    # Compute fidelity with exact state
    psima = ma.to_array()
    overlap = np.abs(np.vdot(psima, psi0))

    print('Epoch = %d   ' % ep,end='')
    print('Overlap = %.6f   '% overlap,end='')
    print("observables={}".format(obs))

Epoch = 10   Overlap = 0.327257   observables={'Energy': (-8.0233 - 0.0000i) ± 0.0062 [var=0.5156, R=0.99979]}
Epoch = 20   Overlap = 0.376776   observables={'Energy': (-8.0692 + 0.0003i) ± 0.0076 [var=0.5423, R=1.00005]}
Epoch = 30   Overlap = 0.428146   observables={'Energy': (-8.1155 + 0.0007i) ± 0.0080 [var=0.5612, R=1.00011]}
Epoch = 40   Overlap = 0.491451   observables={'Energy': (-8.1778 + 0.0009i) ± 0.0063 [var=0.5494, R=0.99978]}
Epoch = 50   Overlap = 0.564125   observables={'Energy': (-8.2760 - 0.0002i) ± 0.0079 [var=0.5185, R=1.00017]}
Epoch = 60   Overlap = 0.665552   observables={'Energy': (-8.3761 + 0.0020i) ± 0.0094 [var=0.4770, R=1.00067]}
Epoch = 70   Overlap = 0.763417   observables={'Energy': (-8.5273 - 0.0009i) ± 0.0095 [var=0.3850, R=1.00109]}
Epoch = 80   Overlap = 0.855221   observables={'Energy': (-8.6480 + 0.0010i) ± 0.0069 [var=0.2699, R=1.00061]}
Epoch = 90   Overlap = 0.916431   observables={'Energy': (-8.7575 + 0.0007i) ± 0.0077 [var=0.1559, R=1.00221]}
E